In [1]:
import time
import requests

class Adapter(requests.adapters.HTTPAdapter):
  def send(self, *args, **kwargs):
    global start
    print(
      "步骤 1 结束，耗时",
      round((time.time() - start) * 1000),
      "毫秒"
    )
    return super().send(*args, **kwargs)

s = requests.Session()
s.mount("https://", Adapter())
start = time.time()
r = s.get('https://www.baidu.com')
end = time.time()
print(
  "步骤 2 结束，耗时",
  round(r.elapsed.total_seconds() * 1000),
  "毫秒"
)
print(
  "步骤 3 结束，耗时",
  int((end -start - r.elapsed.total_seconds()) * 1000),
  "毫秒"
)

步骤 1 结束，耗时 2 毫秒
步骤 2 结束，耗时 447 毫秒
步骤 3 结束，耗时 2 毫秒


In [2]:
import time
import requests
# 导入 concurrent.futures 这个包
from concurrent import futures

urls = ["https://wpblog.x0y1.com/?p=34"] * 30
# 初始化一个线程池，最大的同时任务数是 5
executor = futures.ThreadPoolExecutor(max_workers=5)
session = requests.Session()

fs = []
for url in urls:
  # 提交任务到线程池
  f = executor.submit(session.get, url)
  fs.append(f)

# 等待这些任务全部完成
futures.wait(fs)
# 获取任务的结果
result = [f.result().text for f in fs]

In [3]:
from bs4 import BeautifulSoup

# 解析列表页内容，得到这一页的内容链接
def parse_list_page(text):
  soup = BeautifulSoup(text, 'html.parser')
  tbody = soup.select('table.board-list.tiz > tbody')[0]
  urls = []
  for tr in tbody:
    td = tr.select('td.title_9')[0]
    urls.append(td.a.attrs['href'])
  return urls

In [4]:
# 解析内容页，得到标题和所有帖子内容
def parse_content_page(text):
  soup = BeautifulSoup(text, 'html.parser')

  title = soup.select('span.n-left')[0].text.strip('文章主题:').strip()

  content_div = soup.select('div.b-content.corner')[0]
  contents = []
  for awrap in content_div.select('div.a-wrap.corner'):
    content = awrap.p.text
    contents.append(content)
  return title, contents

In [5]:
def convert_content_url(path):
  URL_PREFIX = 'http://www.mysmth.net'
  path += '?ajax'
  return URL_PREFIX + path

In [6]:
list_urls = []
for i in range(1, 11):
  url = 'http://www.mysmth.net/nForum/board/Python?ajax&p='
  url += str(i)
  list_urls.append(url)

In [7]:
import requests
from concurrent import futures

session = requests.Session()
executor = futures.ThreadPoolExecutor(max_workers=5)

# 这个函数获取列表页数据，解析出链接，并转换成真实链接
def get_content_urls(list_url):
  res = session.get(list_url)
  content_urls = parse_list_page(res.text)
  real_content_urls = []
  for url in content_urls:
    url = convert_content_url(url)
    real_content_urls.append(url)
  return real_content_urls

# 根据刚刚生成的十个列表页链接，开始提交任务
fs = []
for list_url in list_urls:
  fs.append(executor.submit(get_content_urls, list_url))
futures.wait(fs)

content_urls = set()
for f in fs:
  for url in f.result():
    content_urls.add(url)

In [8]:
# 获取正文页内容，解析出标题和帖子
def get_content(url):
  r = session.get(url)
  title, contents = parse_content_page(r.text)
  return title, contents

# 提交解析正文的任务
fs = []
for url in content_urls:
  fs.append(executor.submit(get_content, url))
futures.wait(fs)

results = {}
for f in fs:
  title, contents = f.result()
  results[title] = contents

print(results.keys())

dict_keys(['用Python可以写手机app吗？', 'import的库Unresolved reference 但运行成功是怎么回事？', '请教，pdf文件转excel用什么库好用？', '怎么用import作为参数啊', '循环条件是一个外部函数，且循环体内还需要用到它', '怎么在嵌入python时使用多进程？', '字典怎么赋值？', 'web开发，python还是php?', '有人熟悉aardio+python嘛？', '多进程读log', '谁知道这个提示什么意思', '求助 pandas', '卸载某个包（比如TensorFlow），或者强制升级pip，报这个错，怎', '求教qq音乐抓取的xhr问', 'numpy 里，两个矩阵直接a*b，是什么意思？', '版面积分变更记录', '求推荐Django初学的教材或者视频，以及进阶之路', '伟大的pytorch，将我的识别率提高到了94%，如何改进呢？', '闲鱼上的二手商品历史成交，从哪里能爬到数据阿？', '用手机或者摄像头扫描一行小字条 (转载)\xa0\xa0[查看原文]', '求大佬算一下，这个矩阵的逆矩阵，是多少？谢谢', '突然发现python的logo就是乌克兰国旗', '微信公众号识别用户发来的二维码接口怎么调用？谢谢', '求问一个小白问题逻辑运算', 'D, the Best Programming Language, for Former Python Develop', 'What’s New in PyCharm 2021.3摘要', 'ttkbootstrap给tkinter的仿Bootstrap', '大佬，这种验证码，如何自动识别出来？', '水木Python版推荐使用4个空格作为缩进(转寄) (转载)', 'Python 版版主 hgoldfish 任满卸任 (转载)', '自学完《Python编程:从入门到实践》的第一部分', 'python绘图太慢，有什么办法可以明显提升绘图速度的？', '弱问：random', '本版积分已达积分帽', 'xlrd 的行号，竟然可以是负数？', '有什么版面可以讨论web3.0和nft吗？', 'tf1.5运行，warm_start_from报错，这是什么原因？', '大学学的C++ 没用上 工作后自学pytho